In [ ]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import string
from itertools import islice
from selenium.webdriver.common.keys import Keys
# from Feed import Feed
from icecream import ic
from typing import Iterator, Any, List

In [ ]:
from Surfer import GetBrowser,RedditFeed,Feed
from Surfer.ScriptRunner import ScriptRunner

In [ ]:
driver = GetBrowser()

In [ ]:
def uuid():
    letters_digits = string.ascii_letters
    return ''.join(random.choice(letters_digits) for i in range(20))

In [ ]:
def N(
  mean = 0.5
	, stddev = 0.01
	, bounds = None
):
	if bounds:
		return max(bounds[0], min(random.normalvariate(mean, stddev) , bounds[1]))
	else:
		return random.normalvariate(mean, stddev)

In [48]:
class RedditPostFeed(Feed):
	def __init__(self, driver:webdriver.Chrome ,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Post Feed',driver=driver,handle=handle)
		# Store the WebDriver instance
		self.verbose = verbose
		self.focus()
		self.run_script = ScriptRunner(driver,'./Surfer/RedditBot/scripts/post/')
		self.run_feed_script('listen to clicks')()
		self.run_script('../init_reddit',verbose=self.verbose)()

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not self.run_feed_script('check cursor',verbose=self.verbose)():
				self.run_script('catch first comment',verbose=self.verbose)()
				continue
			self.run_feed_script('scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if self.run_feed_script('check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield self.run_script('parse comment',verbose=self.verbose)()
			self.run_script('cursor next')()

class RedditFeed(Feed):
	def __init__(self, driver:webdriver.Chrome,skip=True,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Feed',handle=handle,driver=driver)
		self.verbose = verbose
		self.run_script = ScriptRunner(driver,'./Surfer/RedditBot/scripts/')
		if not skip:
			self.driver.get('https://www.reddit.com/')
			self.handle = self.driver.current_window_handle
			self.run_feed_script('listen to clicks')()
		self.run_script('init_reddit',verbose=self.verbose)()
	

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not self.run_feed_script('check cursor',verbose=self.verbose)():
				continue
			self.run_feed_script('scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if self.run_feed_script('check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield self.run_script('parse post',verbose=self.verbose)()
			self.run_feed_script('cursor next')()

	def click_post(self, newTab=True):
		for i in range(10): # 10 retries
			self.focus()
			if i == 9:
				raise Exception('cursor not present')
			if not self.run_feed_script('check cursor',verbose=self.verbose)():
				continue
			else:
				break
  
		id = uuid()
		self.run_feed_script('mark cursor', verbose=self.verbose)(id)
		# Find the DOM element to click on
		sel = f'.{id}'
		element = driver.find_element(By.CSS_SELECTOR, sel)
		h3 = element.find_element(By.CSS_SELECTOR,'h3')
		element = element if h3 is None else h3
		actions = ActionChains(driver)
		actions \
			.click(element) \
			.perform()
			# .key_down(Keys.CONTROL) \
			# .key_up(Keys.CONTROL) \
		# self.run_script('catch first comment')()
		return RedditPostFeed(self.driver,self.handle,verbose=self.verbose)

	def close(self):
		self.focus()
		# Close the WebDriver instance
		self.driver.quit()

In [49]:
rf = RedditFeed(driver,skip=False)
# rf = RedditFeed(driver,skip=True)

In [50]:
# for x in islice(rf,15):
#   print(x)
#   time.sleep(2*max(1,len(x) * 0.01))

In [58]:
# run_script('../RedditBot/scripts/init_reddit')()
run_script('../RedditBot/scripts/post/parse comment')()

In [52]:
e = rf.click_post()

In [53]:
for c in e:
	print(c)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [ ]:
e.handle

In [ ]:
driver.current_window_handle

In [ ]:
driver.window_handles

In [ ]:
driver.switch_to.window(driver.window_handles[2])

In [55]:
run_script = ScriptRunner(driver,'./Surfer/scripts/')

In [ ]:
run_script('cluster')()

In [ ]:
d = run_script('post/parse comment')()

In [ ]:
d

In [ ]:
# de-clutter html
# Array.from(x.querySelectorAll('*')).map((x,i)=>{
#     const cl = Array.from(x.classList)
#     const to_remove = cl.filter(x=> !/^[a-z-]+$/.exec(x))
#     for (let cls of to_remove) {
#         x.classList.remove(cls)
#     }
#     if (x.className === ''){
#         x.removeAttribute('class')
#     }
# })